## Run Ollama in Local

In [ ]:
!pip install pypdf==3.12.2
!pip install ollama

In [1]:
import ollama
import fitz

### Talk to GenAI

In [2]:
# model = 'llama2' #7 billion
# model = 'llama3.2' #3 billion
# model = 'llama3.1:70b' #70 billion
model = 'llama3.1:8b' #8 billion

In [3]:
messages = [
    {
        'role': 'user',
        'content': 'Kenapa langit berwarna biru?'
    }
]

response_stream = ollama.chat(model=model, messages=messages, stream=True)

for chunk in response_stream:
    print(chunk['message']['content'], end='', flush=True)

Langit terlihat biru karena cahaya matahari menyebar dalam gelombang cahaya, yang kemudian dipancarkan ke segala penjuru dengan warna putih. Kemudian, cahaya tersebut mencapai atmosfer Bumi. Di sana, cahaya digabung oleh molekul gas (terutama oksigen dan nitrogen) dan menghasilkan sinar biru.

### Talk with your document!

In [30]:
def process_pdf(pdf_path):

    try:
        # Load PDF silently
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        doc.close()
        
        return text  # Return the extracted text instead of creating a document index

    except Exception as e:
        print(f"Error processing PDF: {str(e)}")
        return None

def get_clean_response(model, query):
    messages = [
        {
            'role': 'user',
            'content': query
        }
    ]
    
    response_stream = ollama.chat(model=model, messages=messages, stream=True)
    
    # Stream the response
    for chunk in response_stream:
        if 'message' in chunk and 'content' in chunk['message']:
            yield chunk['message']['content']  # Yield each chunk of the response


In [31]:
# pdf_path = "/Users/adhityaraar/Documents/IBM/project/RAG-OS/data/Simmons_Bank.pdf"
pdf_path = "/Users/adhityaraar/Documents/GitHub/GA-AI-Workshop/data/Peraturan_Perusahaan.pdf"
extracted_text = process_pdf(pdf_path)

In [27]:
# model = 'llama2' #7 billion
# model = 'llama3.2' #3 billion
# model = 'llama3.1:70b' #70 billion
model = 'llama3.1:8b' #8 billion

In [32]:
# user_question = "Tolong jelaskan apa yang di maksud dalam dokumen tersebut"
user_question = "Berapa usia pensiun karyawan?"

if extracted_text:
    query = f"{user_question}: {extracted_text}"  # Limit for context
    model = 'llama3.2'

    # Print the response as it streams
    for response_chunk in get_clean_response(model, query):
        print(response_chunk, end='', flush=True)

Usia pensiun karyawan adalah 60 tahun.

### Talk with your document with Prompt!

In [ ]:
def get_clean_response(model, context, query):
    # Define the prompt template
    prompt_template = """ 
        Anda adalah asisten yang membantu user dalam menjawab pertanyaan terkait suatu dokumen. Tugas utama Anda adalah menganalisis pertanyaan dari pengguna dan hasil query yang telah disediakan, kemudian memberikan jawaban yang jelas, tepat, dan bermanfaat.

        Pertanyaan: {query}
        Konteks: {context}

        Instruksi bagaimana cara menjawab:
        1. Pastikan jawaban Anda menyertakan penjelasan yang sesuai dengan konteks. 
        2. Jika data kosong, katakan bahwa Anda tidak dapat mendapatkan datanya sehingga pertanyaan tidak dapat dijawab dan coba untuk bertanya lebih spesifik terkait data transaksi.
        3. Hasil jawaban yang disediakan berisi data penting yang perlu dianalisis untuk memberikan jawaban yang akurat. Jika pertanyaan menyangkut jumlah finansial atau uang, formatkan angka tersebut dalam format mata uang Rupiah dengan desimal yang sesuai.
        4. Jika Anda tidak dapat memberikan jawaban, jawab dengan sopan bahwa Anda tidak dapat menjawabnya.
        5. Jika jawaban yang diberikan menggunakan huruf kapital semua, ubah formatnya mengikuti kaidah bahasa Indonesia yang benar.
        6. Jawaban harus menggunakan Bahasa Indonesia yang baik dan benar, hindari penggunaan tanda petik dan hindari penggunaan tanda baris baru.

        Jawaban: 
    """
    # Insert context and query into the prompt
    prompt = prompt_template.format(context=context, query=query)

    # Pass prompt into Ollama chat function
    messages = [
        {
            'role': 'user',
            'content': prompt
        }
    ]
    
    response_stream = ollama.chat(model=model, messages=messages, stream=True)
    
    # Stream the response
    for chunk in response_stream:
        if 'message' in chunk and 'content' in chunk['message']:
            yield chunk['message']['content']  # Yield each chunk of the response

In [ ]:
pdf_path = "/Users/adhityaraar/Documents/GitHub/GA-AI-Workshop/data/Peraturan_Perusahaan.pdf"
extracted_text = process_pdf(pdf_path)

In [34]:
# user_question = "Pada pukul berapa karyawan mulai bekerja?"
user_question = "Berapa usia pensiun karyawan?"
# user_question = "Berapa lama masa percobaan karyawan baru?"

# user_question = "Kapan gaji akan dibayarkan jika tanggal 25 jatuh pada hari Sabtu?"
# user_question = "Apa yang harus dilakukan karyawan jika ingin menggunakan hak cuti tahunannya?"
# user_question = "pakaian apa yang digunakan pada hari Selasa?"

# user_question = "Apa saja tunjangan yang dapat diterima karyawan yang telah bekerja minimal 12 bulan di perusahaan, dan bagaimana aturan terkait Tunjangan Hari Raya (THR) untuk karyawan yang berhenti bekerja paling lama 30 hari sebelum hari raya Idul Fitri?"
# user_question = "Bagaimana perusahaan mengevaluasi performa karyawan baru selama masa percobaan, dan siapa yang terlibat dalam proses pengawasan dan penilaian?"
# user_question = "Bicarakan mengenai jaminan sosial karyawan yang diberikan oleh perusahaan, termasuk hak karyawan yang meninggal dan hak keluarga karyawan yang meninggal dalam hal jaminan sosial."
# user_question = "apa itu ayam?"

if extracted_text:
    query = f"{user_question}"  # Limit for context

    # Print the response as it streams
    for response_chunk in get_clean_response(model, extracted_text, query):
        print(response_chunk, end='', flush=True)

Berdasarkan konteks peraturan perusahaan yang disediakan, jawaban untuk pertanyaan "Berapa usia pensiun karyawan?" adalah:

"Usia pensiun karyawan adalah 60 tahun."